# Конфигурация

In [1]:
import pickle
from collections import Counter
import random

import pandas as pd
import numpy as np
from scipy import sparse

from tqdm.auto import tqdm

import gensim.downloader as api

import rectools 
from rectools.model_selection import TimeRangeSplitter
from rectools.metrics import MAP, NDCG, Precision, Recall, MeanInvUserFreq, Serendipity, calc_metrics
from rectools.tools.ann import UserToItemAnnRecommender

import tensorflow.keras.backend as K
from tensorflow import keras

from sklearn.metrics.pairwise import euclidean_distances, cosine_distances, cosine_similarity

In [2]:
K_RECOS = 10
RANDOM_SEED = 42

np.random.seed(RANDOM_SEED)

In [3]:
metrics = {'MAP@1': MAP(k=1), 'MAP@5': MAP(k=5), 'MAP@10': MAP(k=10),
           'NDCG@1': NDCG(k=1), 'NDCG@5': NDCG(k=5), 'NDCG@10': NDCG(k=10),
           'Precision@1': Precision(k=1), 'Precision@5': Precision(k=5), 'Precision@10': Precision(k=10),
           'Recall@1': Recall(k=1), 'Recall@5': Recall(k=5), 'Recall@10': Recall(k=10),
           'MeanInvUserFreq@1': MeanInvUserFreq(k=1), 'MeanInvUserFreq@5': MeanInvUserFreq(k=5), 'MeanInvUserFreq@10': MeanInvUserFreq(k=10),
           'Serendipity@1': Serendipity(k=1), 'Serendipity@5': Serendipity(k=5), 'Serendipity@10': Serendipity(k=10)}

# Загрузка и обработка данных

In [4]:
users = pd.read_csv('data_original/users.csv')
items = pd.read_csv('data_original/items.csv')
interactions = pd.read_csv('data_original/interactions.csv', parse_dates=["last_watch_dt"])

## Обработка юзеров

In [5]:
users.fillna('unknown', inplace=True)

In [6]:
users['age'] = users['age'].astype('category')

In [7]:
users['income'] = users['income'].astype('category')

In [8]:
users['sex'] = users['sex'].astype('category')

In [9]:
users['kids_flg'] = users['kids_flg'].astype('bool')

## Обработка айтемов

### Тип контента

In [10]:
items['content_type'] = items['content_type'].astype('category')

### Заголовок

In [11]:
items['title'] = items['title'].str.lower()

In [12]:
items['title_orig'] = items['title_orig'].fillna('unknown') 

### Год выпуска

In [13]:
items['release_year'] = items['release_year'].fillna(2020) 

In [14]:
items.loc[items['release_year'] < 1920, 'release_year_cat'] = 'inf_1920'
items.loc[items['release_year'] >= 2020, 'release_year_cat'] = '2020_inf'

In [15]:
for i in range (1920, 2020, 10):
    items.loc[(items['release_year'] >= i) & (items['release_year'] < i+10), 'release_year_cat'] = f'{i}-{i+10}'

In [16]:
items = items.drop(columns=['release_year'])
items['release_year_cat'] = items['release_year_cat'].astype('category')

### Жанр

In [17]:
items['genres'] = items['genres'].astype('category')

### Страна

In [18]:
items['countries'] = items['countries'].fillna('Россия') 

In [19]:
items['countries'] = items['countries'].str.lower()
items['countries'] = items['countries'].apply(lambda x: ', '.join(sorted(list(set(x.split(', '))))))

In [20]:
items['countries'] = items['countries'].astype('category')

### Возрастные ограничения

In [21]:
items['for_kids'] = items['for_kids'].fillna(0)

In [22]:
items['for_kids'] = items['for_kids'].astype('bool')

In [23]:
items['age_rating'] = items['age_rating'].fillna(0) 

In [24]:
items['age_rating'] = items['age_rating'].astype('category')

### Студия

In [25]:
items['studios'] = items['studios'].fillna('unknown')

In [26]:
items['studios'] = items['studios'].str.lower()
items['studios'] = items['studios'].apply(lambda x: ', '.join(sorted(list(set(x.split(', '))))))

In [27]:
items['studios'] = items['studios'].astype('category')

### Директор

In [28]:
items['directors'] = items['directors'].fillna('unknown')

In [29]:
items['directors'] = items['directors'].str.lower()

In [30]:
items['directors'] = items['directors'].astype('category')

### Актёры

In [31]:
items['actors'] = items['actors'].fillna('unknown')

In [32]:
items['actors'] = items['actors'].astype('category')

### Ключевые слова

In [33]:
items['keywords'] = items['keywords'].fillna('unknown')

In [34]:
items['keywords'] = items['keywords'].apply(lambda x: list(x.lower().replace(',','').split()))

### Описание

In [35]:
items['description'] = items['description'].fillna('unknown')

## Обработка взаимодействий

In [36]:
interactions['watched_pct'] = interactions['watched_pct'].astype(pd.Int8Dtype())

In [37]:
interactions['watched_pct'] = interactions['watched_pct'].fillna(0)

# Подготовка данных

В качестве фичей юзеров были выбраны: возраст, доход, пол, наличие детей

In [38]:
user_cat_feats = ["age", "income", "sex", "kids_flg"]
users_ohe = users.user_id
for feat in user_cat_feats:
    # получаем датафрейм с one-hot encoding для каждой категориальной фичи
    ohe_feat = pd.get_dummies(users[feat], prefix=feat)
    # конкатенируем ohe-hot датафрейм с датафреймом, 
    # который мы получили на предыдущем шаге
    users_ohe = pd.concat([users_ohe, ohe_feat], axis=1)

users_ohe.head()

,user_id,age_age_18_24,age_age_25_34,age_age_35_44,age_age_45_54,age_age_55_64,age_age_65_inf,age_unknown,income_income_0_20,income_income_150_inf,income_income_20_40,income_income_40_60,income_income_60_90,income_income_90_150,income_unknown,sex_unknown,sex_Ж,sex_М,kids_flg_False,kids_flg_True
0,973171,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1
1,962099,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0
2,1047345,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0
3,721985,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0
4,704055,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0


In [39]:
print('Number user features:', len(users_ohe.columns)-1)

Number user features: 19


In [40]:
users_ohe["uid"] = users_ohe["user_id"].astype("category")
users_ohe["uid"] = users_ohe["uid"].cat.codes

uid_to_user_id = users_ohe[["uid", "user_id"]].to_dict()["user_id"]
user_id_to_uid = {v:k for k, v in zip(uid_to_user_id.keys(), uid_to_user_id.values())}

users_ohe.drop(columns=["uid"], inplace=True)

In [62]:
# with open('user_id_to_uid.pkl', 'wb') as file:
#     pickle.dump(user_id_to_uid, file)

In [63]:
# users_ohe.to_pickle('users_features_dssm.pkl')

В качестве фичей айтемов были выбраны: тип контента, диапазон даты выхода, флаг "для детей", возрастной рейтинг, студия, страна, директор

In [64]:
item_cat_feats = ['content_type', 'release_year_cat',
                  'for_kids', 'age_rating', 
                  'studios', 'countries', 'directors']

items_ohe = items.item_id

for feat in item_cat_feats:
    ohe_feat = pd.get_dummies(items[feat], prefix=feat)
    items_ohe = pd.concat([items_ohe, ohe_feat], axis=1) 

А также подготовлены текстовые фичи из word2vec

In [65]:
wv = api.load('word2vec-ruscorpora-300')

In [66]:
# убираем теги из словаря word2vec
vectors = np.zeros((len(wv.index_to_key), wv.vector_size))
new_index = 0
word_to_index = {}
for i, word_with_tag in enumerate(wv.index_to_key):
    word, tag = word_with_tag.split('_')
    if word not in word_to_index:
        word_to_index[word] = new_index
        new_index += 1
        vectors[word_to_index[word]] = wv.vectors[i]
wv.index_to_key = list(word_to_index.keys())
wv.key_to_index = {word: index for index, word in enumerate(wv.index_to_key)}
wv.vectors = vectors

In [67]:
# получаем вектора для айтемов как сумму их ключевых слов из word2vec
embedding_matrix = np.zeros((items.shape[0], wv.vector_size))
for i in range(items.shape[0]):
    embedding_matrix[i] = wv.get_mean_vector(items.loc[i, 'keywords']).copy()

In [68]:
items_ohe = pd.concat([items_ohe, pd.DataFrame(embedding_matrix).add_prefix('word2vec_')], axis=1) 

In [69]:
items_ohe.head()

,item_id,content_type_film,content_type_series,release_year_cat_1920-1930,release_year_cat_1930-1940,release_year_cat_1940-1950,release_year_cat_1950-1960,release_year_cat_1960-1970,release_year_cat_1970-1980,release_year_cat_1980-1990,...,word2vec_290,word2vec_291,word2vec_292,word2vec_293,word2vec_294,word2vec_295,word2vec_296,word2vec_297,word2vec_298,word2vec_299
0,10711,1,0,0,0,0,0,0,0,0,...,0.036104,0.030113,0.001830,0.026430,-0.009028,-0.000474,0.019233,0.063747,-0.010099,0.046014
1,2508,1,0,0,0,0,0,0,0,0,...,0.033976,0.034263,0.013838,0.015138,-0.033904,-0.045648,0.062685,0.065262,0.001389,0.030032
2,10716,1,0,0,0,0,0,0,0,0,...,0.030757,-0.003070,-0.017414,0.020658,-0.017720,-0.016413,0.050801,0.050227,-0.038631,-0.000711
3,7868,1,0,0,0,0,0,0,0,0,...,0.029124,0.004448,-0.011717,0.025576,-0.020309,-0.041174,0.029119,0.037307,0.003491,0.044989
4,16268,1,0,0,0,0,0,0,1,0,...,0.002734,-0.018169,-0.031675,0.007064,-0.011285,-0.038776,0.014847,0.058526,-0.033943,0.020064


In [70]:
del wv, vectors, embedding_matrix, word_to_index

In [71]:
print('Number item features:', len(items_ohe.columns)-1)

Number item features: 8888


# Фильтрация данных

В датасете взаимодействий есть непопулярные фильмы и малоактивные пользователи. Кроме того, в таблице взаимодействий есть события с низким качеством взаимодействия - когда юзер начал смотреть фильм, но вскоре после начала просмотра выключил.

Отфильтруем такие события, малоактивных юзеров и непопулярные фильмы.

In [72]:
print(f"N users before: {interactions.user_id.nunique()}")
print(f"N items before: {interactions.item_id.nunique()}\n")

# отфильтруем все события взаимодействий, в которых пользователь посмотрел
# фильм менее чем на 10 процентов
interactions = interactions[interactions.watched_pct > 10]

# соберем всех пользователей, которые посмотрели 
# больше 10 фильмов
valid_users = []

c = Counter(interactions.user_id)
for user_id, entries in c.most_common():
    if entries > 10:
        valid_users.append(user_id)

# и соберем все фильмы, которые посмотрели больше 3 пользователей
valid_items = []

c = Counter(interactions.item_id)
for item_id, entries in c.most_common():
    if entries > 3:
        valid_items.append(item_id)

# отбросим непопулярные фильмы и неактивных юзеров
interactions = interactions[interactions.user_id.isin(valid_users)]
interactions = interactions[interactions.item_id.isin(valid_items)]

print(f"N users after: {interactions.user_id.nunique()}")
print(f"N items after: {interactions.item_id.nunique()}")

N users before: 962179
N items before: 15706

N users after: 79515
N items after: 9387


После фильтрации может получиться так, что некоторые айтемы/юзеры есть в датасете взаимодействий, но при этом они отсутствуют в датасетах айтемов/юзеров или наоборот. Поэтому найдем id айтемов и id юзеров, которые есть во всех датасетах и оставим только их.

In [73]:
common_users = set(interactions.user_id.unique()).intersection(set(users_ohe.user_id.unique()))

interactions = interactions[interactions.user_id.isin(common_users)]
users_ohe = users_ohe[users_ohe.user_id.isin(common_users)]

common_items = set(interactions.item_id.unique()).intersection(set(items_ohe.item_id.unique()))

interactions = interactions[interactions.item_id.isin(common_items)]
items_ohe = items_ohe[items_ohe.item_id.isin(common_items)]

print(len(common_users))
print(len(common_items))

65974
9293


# Разделение данных

Основная сложность в разделении данных заключается в сопоставлении индексов в train и test

In [74]:
max_date = interactions['last_watch_dt'].max()

interactions_train = interactions[(interactions['last_watch_dt'] < max_date - pd.Timedelta(days=7))].copy()
users_ohe_train = users_ohe[users_ohe['user_id'].isin(interactions_train['user_id'].unique())].copy()
items_ohe_train = items_ohe[items_ohe['item_id'].isin(interactions_train['item_id'].unique())].copy()

interactions_test = interactions[(interactions['last_watch_dt'] >= max_date - pd.Timedelta(days=7))].copy()

# оставляем только теплых пользователей в тесте
interactions_hot_test = interactions_test[(interactions_test['user_id'].isin(interactions_train['user_id'].unique())) & (interactions_test['item_id'].isin(interactions_train['item_id'].unique()))].copy()
users_ohe_hot_test = users_ohe[users_ohe['user_id'].isin(interactions_hot_test['user_id'].unique())].copy()

catalog = interactions_train['item_id'].unique()

print(f"train: {interactions_train.shape}")
print(f"test: {interactions_test.shape}")
print(f"hot test: {interactions_hot_test.shape}")

train: (1368251, 5)
test: (103085, 5)
hot test: (97278, 5)


In [75]:
# items_ohe_train.to_pickle('items_features_dssm.pkl')

In [76]:
del interactions, users_ohe, items_ohe

Соберем взаимодействия в матрицу user*item так, чтобы в строках этой матрицы были user_id, в столбцах - item_id, а на пересечениях строк и столбцов - единица, если пользователь взаимодействовал с айтемом и ноль, если нет.

Такую матрицу удобно собирать в numpy array, однако нужно помнить, что numpy array индексируется порядковыми индексами, а нам же удобнее использовать item_id и user_id.

Создадим внутренние индексы для user_id и item_id - uid и iid. Для этого просто соберем все user_id и item_id и пронумеруем их по порядку.

In [77]:
interactions_train["uid"] = interactions_train["user_id"].astype("category")
interactions_train["uid"] = interactions_train["uid"].cat.codes

interactions_hot_test["uid"] = interactions_hot_test["user_id"].astype("category")
interactions_hot_test["uid"] = interactions_hot_test["uid"].cat.codes

interactions_train["iid"] = interactions_train["item_id"].astype("category")
interactions_train["iid"] = interactions_train["iid"].cat.codes

mapping_iid = dict(zip(interactions_train['item_id'], interactions_train['iid']))
interactions_hot_test['iid'] = interactions_hot_test['item_id'].map(mapping_iid)
del mapping_iid

print(sorted(interactions_train.iid.unique())[:5])
print(sorted(interactions_train.uid.unique())[:5])
print(sorted(interactions_hot_test.iid.unique())[:5])
print(sorted(interactions_hot_test.uid.unique())[:5])

[0, 1, 2, 3, 4]
[0, 1, 2, 3, 4]
[0, 2, 3, 6, 7]
[0, 1, 2, 3, 4]


In [78]:
interactions_train.head()

,user_id,item_id,last_watch_dt,total_dur,watched_pct,uid,iid
0,176549,9506,2021-05-11,4250,72,10562,5297
1,699317,1659,2021-05-29,8317,100,41956,927
6,1016458,354,2021-08-14,1672,25,60778,184
7,884009,693,2021-08-04,703,14,52930,376
14,5324,8437,2021-04-18,6598,92,309,4701


In [79]:
interactions_hot_test.head()

,user_id,item_id,last_watch_dt,total_dur,watched_pct,uid,iid
64,73446,14488,2021-08-19,6011,100,1830,8093
141,626036,11109,2021-08-22,1323,19,15738,6189
225,45247,7135,2021-08-18,5229,83,1147,3987
360,962127,9617,2021-08-17,1910,23,24307,5354
366,217842,5680,2021-08-22,7494,100,5460,3175


In [80]:
print(interactions_train.item_id.nunique())
print(items_ohe_train.item_id.nunique())

print(interactions_train.user_id.nunique())
print(users_ohe_train.user_id.nunique())

print(interactions_hot_test.user_id.nunique())
print(users_ohe_hot_test.user_id.nunique())

print(set(items_ohe_train.item_id.unique()) - set(interactions_train.item_id.unique()))

9221
9221
65711
65711
27715
27715
set()


Для того, чтобы можно было удобно превратить iid/uid в item_id/user_id и наоборот соберем словари 

{iid: item_id}, {uid: user_id} и {item_id: iid}, {user_id: uid}.

In [81]:
iid_to_item_id_train = interactions_train[["iid", "item_id"]].drop_duplicates().set_index("iid").to_dict()["item_id"]
item_id_to_iid_train = interactions_train[["iid", "item_id"]].drop_duplicates().set_index("item_id").to_dict()["iid"]

uid_to_user_id_train = interactions_train[["uid", "user_id"]].drop_duplicates().set_index("uid").to_dict()["user_id"]
user_id_to_uid_train = interactions_train[["uid", "user_id"]].drop_duplicates().set_index("user_id").to_dict()["uid"]

uid_to_user_id_hot_test = interactions_hot_test[["uid", "user_id"]].drop_duplicates().set_index("uid").to_dict()["user_id"]
user_id_to_uid_hot_test = interactions_hot_test[["uid", "user_id"]].drop_duplicates().set_index("user_id").to_dict()["uid"]

In [82]:
# with open('iid_to_item_id.pkl', 'wb') as file:
#     pickle.dump(iid_to_item_id_train, file)

In [83]:
# with open('item_id_to_iid.pkl', 'wb') as file:
#     pickle.dump(item_id_to_iid_train, file)

И проиндексируем датасеты users_ohe и items_ohe по внутренним айди:

In [84]:
items_ohe_train["iid"] = items_ohe_train["item_id"].apply(lambda x: item_id_to_iid_train[x])
items_ohe_train = items_ohe_train.set_index("iid")

users_ohe_train["uid"] = users_ohe_train["user_id"].apply(lambda x: user_id_to_uid_train[x])
users_ohe_train = users_ohe_train.set_index("uid")

users_ohe_hot_test["uid"] = users_ohe_hot_test["user_id"].apply(lambda x: user_id_to_uid_hot_test[x])
users_ohe_hot_test = users_ohe_hot_test.set_index("uid")

In [85]:
items_ohe_train.sort_index(inplace=True)
users_ohe_train.sort_index(inplace=True)
users_ohe_hot_test.sort_index(inplace=True)

In [86]:
items_ohe_train.head()

,item_id,content_type_film,content_type_series,release_year_cat_1920-1930,release_year_cat_1930-1940,release_year_cat_1940-1950,release_year_cat_1950-1960,release_year_cat_1960-1970,release_year_cat_1970-1980,release_year_cat_1980-1990,...,word2vec_290,word2vec_291,word2vec_292,word2vec_293,word2vec_294,word2vec_295,word2vec_296,word2vec_297,word2vec_298,word2vec_299
iid,,,,,,,,,,,,,,,,,,,,,
0,0,0,1,0,0,0,0,0,0,0,...,0.010037,0.025415,0.005816,-0.003805,-0.005076,-0.032212,0.036976,0.024176,0.018876,0.036757
1,1,1,0,0,0,0,0,0,0,0,...,0.009541,0.051739,0.006029,0.016854,-0.051090,0.017357,0.021642,0.024663,-0.016797,0.035952
2,2,1,0,0,0,0,0,0,0,0,...,0.068823,0.019328,0.038019,-0.017185,-0.000445,0.003444,0.004976,0.021308,0.021809,0.056520
3,3,1,0,0,0,0,0,0,0,0,...,0.015820,0.013845,-0.010117,0.014662,-0.017577,-0.012337,0.007987,0.028970,-0.027093,0.020757
4,4,1,0,0,0,0,0,0,0,0,...,0.003355,0.009198,-0.004302,0.030392,-0.018730,-0.059582,0.027593,0.034496,0.018268,0.043874


In [87]:
users_ohe_train.head()

,user_id,age_age_18_24,age_age_25_34,age_age_35_44,age_age_45_54,age_age_55_64,age_age_65_inf,age_unknown,income_income_0_20,income_income_150_inf,income_income_20_40,income_income_40_60,income_income_60_90,income_income_90_150,income_unknown,sex_unknown,sex_Ж,sex_М,kids_flg_False,kids_flg_True
uid,,,,,,,,,,,,,,,,,,,,
0,2,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1
1,21,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0
2,53,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0
3,60,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1
4,81,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0


In [88]:
users_ohe_hot_test.head()

,user_id,age_age_18_24,age_age_25_34,age_age_35_44,age_age_45_54,age_age_55_64,age_age_65_inf,age_unknown,income_income_0_20,income_income_150_inf,income_income_20_40,income_income_40_60,income_income_60_90,income_income_90_150,income_unknown,sex_unknown,sex_Ж,sex_М,kids_flg_False,kids_flg_True
uid,,,,,,,,,,,,,,,,,,,,
0,21,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0
1,53,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0
2,241,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0
3,321,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0
4,322,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0


## Использовать информацию о качестве взаимодействия юзеров с айтемами для более репрезентативного сэмплирования (пункт 1 задание 1) (3 балла)

В качестве информации использовался процент просмотренного, таким образом более просмотренные айтемы имеют большую вероятность быть выбранными, но вычисление вероятностей замедлилось

Также пробовал использовать количество просмотренных минут, результаты получились лучше, но создание вектора замедляется ещё в 2 раза

In [89]:
%%time
interactions_vec_train = np.zeros((interactions_train.uid.nunique(), 
                                   interactions_train.iid.nunique())) 

for i, (user_id, item_id) in enumerate(zip(interactions_train.uid, interactions_train.iid)):
    # interactions_vec_train[user_id, item_id] += interactions_train.watched_pct.iat[i] * interactions_train.total_dur.iat[i]
    interactions_vec_train[user_id, item_id] += interactions_train.watched_pct.iat[i]

res = interactions_vec_train.sum(axis=1)
for i in range(len(interactions_vec_train)):
    interactions_vec_train[i] /= res[i]

CPU times: total: 27.9 s
Wall time: 29.5 s


In [90]:
%%time
interactions_vec_hot_test = np.zeros((interactions_hot_test.uid.nunique(), 
                                      interactions_train.iid.nunique())) 

for i, (user_id, item_id) in enumerate(zip(interactions_hot_test.uid, interactions_hot_test.iid)):
    # interactions_vec_hot_test[user_id, item_id] += interactions_hot_test.watched_pct.iat[i] * interactions_hot_test.total_dur.iat[i]
    interactions_vec_hot_test[user_id, item_id] += interactions_hot_test.watched_pct.iat[i]

res = interactions_vec_hot_test.sum(axis=1)
for i in range(len(interactions_vec_hot_test)):
    interactions_vec_hot_test[i] /= res[i]

CPU times: total: 3.19 s
Wall time: 3.5 s


## Генератор и семплирование

Сделаем простой генератор. Он будет брать рандромного юзера, и два разных айтема - хороший пример и плохой:
- хорошим примером будет тот айтем, который был взят из датасета взаимодействий в соответствии с распределением просмотренных айтемов для этого юзера;
- а плохим айтемом будет просто любой другой _случайный айтем_*


In [64]:
def generator(items, users, interactions, batch_size=16):
    while True:
        uid_meta = []
        uid_interaction = []
        pos = []
        neg = []
        for _ in range(batch_size):
            
            # берем рандомный uid
            uid_i = random.randint(0, interactions.shape[0]-1)
            
            # iid хорошего айтема
            pos_i = np.random.choice(range(interactions.shape[1]), p=interactions[uid_i])
            
            # iid плохого айтема
            neg_i = np.random.choice(range(interactions.shape[1]))
            
            # фичи юзера
            uid_meta.append(users.loc[uid_i])
            
            # вектор айтемов, с которыми юзер взаимодействовал
            uid_interaction.append(interactions[uid_i])
            
            # фичи хорошего айтема
            pos.append(items.loc[pos_i])
            
            # фичи плохого айтема
            neg.append(items.loc[neg_i])
            
        yield [np.array(uid_meta), np.array(uid_interaction), np.array(pos), np.array(neg)], [np.array(uid_meta), np.array(uid_interaction)]

In [65]:
# инициализируем генератор
gen = generator(items=items_ohe_train.drop(["item_id"], axis=1), 
                users=users_ohe_train.drop(["user_id"], axis=1), 
                interactions=interactions_vec_train,
                batch_size=2)

ret = next(gen)

print(f"вектор фичей юзера: {ret[0][0].shape}")
print(f"вектор взаимодействий юзера с айтемами: {ret[0][1].shape}")
print(f"вектор 'хорошего' айтема: {ret[0][2].shape}")
print(f"вектор 'плохого' айтема: {ret[0][3].shape}")
print()
print(f"вектор фичей юзера: {ret[1][0].shape}")
print(f"вектор взаимодействий юзера с айтемами: {ret[1][1].shape}")
del ret

вектор фичей юзера: (2, 19)
вектор взаимодействий юзера с айтемами: (2, 9221)
вектор 'хорошего' айтема: (2, 8888)
вектор 'плохого' айтема: (2, 8888)

вектор фичей юзера: (2, 19)
вектор взаимодействий юзера с айтемами: (2, 9221)


In [91]:
N_FACTORS = 512

# в датасетах есть столбец user_id/item_id, помним, что он не является фичей для обучения!
ITEM_MODEL_SHAPE = (items_ohe_train.drop(["item_id"], axis=1).shape[1], ) 
USER_META_MODEL_SHAPE = (users_ohe_train.drop(["user_id"], axis=1).shape[1], )

USER_INTERACTION_MODEL_SHAPE = (interactions_vec_train.shape[1], )

print(f"N_FACTORS: {N_FACTORS}")
print(f"ITEM_MODEL_SHAPE: {ITEM_MODEL_SHAPE}")
print(f"USER_META_MODEL_SHAPE: {USER_META_MODEL_SHAPE}")
print(f"USER_INTERACTION_MODEL_SHAPE: {USER_INTERACTION_MODEL_SHAPE}")

N_FACTORS: 512
ITEM_MODEL_SHAPE: (8888,)
USER_META_MODEL_SHAPE: (19,)
USER_INTERACTION_MODEL_SHAPE: (9221,)


# Модель

## Инициализация

In [92]:
def triplet_loss(y_true, y_pred, n_dims=N_FACTORS, alpha=0.4):
    # будем ожидать, что на вход функции прилетит три сконкатенированных 
    # вектора - вектор юзера и два вектора айтема
    anchor = y_pred[:, 0:n_dims]
    positive = y_pred[:, n_dims:n_dims*2]
    negative = y_pred[:, n_dims*2:n_dims*3]

    # считаем расстояния от вектора юзера до вектора хорошего айтема
    pos_dist = K.sum(K.square(anchor - positive), axis=1)
    # и до плохого
    neg_dist = K.sum(K.square(anchor - negative), axis=1)

    # считаем лосс
    basic_loss = pos_dist - neg_dist + alpha
    loss = K.maximum(basic_loss, 0.0) # возвращаем ноль, если лосс отрицательный
 
    return loss

In [68]:
def item_model(n_factors=N_FACTORS):
    # входной слой
    inp = keras.layers.Input(shape=ITEM_MODEL_SHAPE)
    
    # полносвязный слой
    layer_1 = keras.layers.Dense(N_FACTORS, activation='elu', use_bias=False,
                                 kernel_regularizer=keras.regularizers.l2(1e-6),
                                 activity_regularizer=keras.regularizers.l2(l2=1e-6))(inp)

    # делаем residual connection - складываем два слоя, 
    # чтобы градиенты не затухали во время обучения
    layer_2 = keras.layers.Dense(N_FACTORS, activation='elu', use_bias=False,
                                 kernel_regularizer=keras.regularizers.l2(1e-6),
                                 activity_regularizer=keras.regularizers.l2(l2=1e-6))(layer_1)
    
    add = keras.layers.Add()([layer_1, layer_2])
    
    # выходной слой
    out = keras.layers.Dense(N_FACTORS, activation='linear', use_bias=False,
                             kernel_regularizer=keras.regularizers.l2(1e-6),
                             activity_regularizer=keras.regularizers.l2(l2=1e-6))(add)
    
    return keras.models.Model(inp, out)


def user_model(n_factors=N_FACTORS):
    
    # входной слой для вектора фичей юзера (из users_ohe)
    inp_meta = keras.layers.Input(shape=USER_META_MODEL_SHAPE)
    
    # входной слой для вектора просмотров (из iteractions_vec)
    inp_interaction = keras.layers.Input(shape=USER_INTERACTION_MODEL_SHAPE)

    # полносвязный слой
    layer_1_meta = keras.layers.Dense(N_FACTORS, activation='elu', use_bias=False,
                                 kernel_regularizer=keras.regularizers.l2(1e-6),
                                 activity_regularizer=keras.regularizers.l2(l2=1e-6))(inp_meta)

    layer_1_interaction = keras.layers.Dense(N_FACTORS, activation='elu', use_bias=False,
                                 kernel_regularizer=keras.regularizers.l2(1e-6),
                                 activity_regularizer=keras.regularizers.l2(l2=1e-6))(inp_interaction)

    # делаем residual connection - складываем два слоя,
    # чтобы градиенты не затухали во время обучения
    layer_2_meta = keras.layers.Dense(N_FACTORS, activation='elu', use_bias=False,
                                 kernel_regularizer=keras.regularizers.l2(1e-6),
                                 activity_regularizer=keras.regularizers.l2(l2=1e-6))(layer_1_meta)
    

    add = keras.layers.Add()([layer_1_meta, layer_2_meta])
    
    # конкатенируем вектор фичей с вектором просмотров
    concat_meta_interaction = keras.layers.Concatenate()([add, layer_1_interaction])
    
    # выходной слой
    out = keras.layers.Dense(N_FACTORS, activation='linear', use_bias=False,
                             kernel_regularizer=keras.regularizers.l2(1e-6),
                             activity_regularizer=keras.regularizers.l2(l2=1e-6))(concat_meta_interaction)
    
    return keras.models.Model([inp_meta, inp_interaction], out)

In [69]:
# инициализируем модели юзера и айтема
i2v = item_model()
u2v = user_model()

# вход для вектора фичей юзера (из users_ohe)
ancor_meta_in = keras.layers.Input(shape=USER_META_MODEL_SHAPE)

# вход для вектора просмотра юзера (из interactions_vec)
ancor_interaction_in = keras.layers.Input(shape=USER_INTERACTION_MODEL_SHAPE)

# вход для вектора "хорошего" айтема
pos_in = keras.layers.Input(shape=ITEM_MODEL_SHAPE)

# вход для вектора "плохого" айтема
neg_in = keras.layers.Input(shape=ITEM_MODEL_SHAPE)

# получаем вектор юзера
ancor = u2v([ancor_meta_in, ancor_interaction_in]) # model_1

# получаем вектор "хорошего" айтема
pos = i2v(pos_in) # model

# получаем вектор "плохого" айтема
neg = i2v(neg_in) # model

# конкатенируем полученные векторы
res = keras.layers.Concatenate(name="concat_ancor_pos_neg")([ancor, pos, neg])

# собираем модель
model = keras.models.Model([ancor_meta_in, ancor_interaction_in, pos_in, neg_in], res)

In [70]:
model_name = 'recsys_resnet_linear'

# логируем процесс обучения в тензорборд
t_board = keras.callbacks.TensorBoard(log_dir=f'runs/{model_name}')

# уменьшаем learning_rate, если лосс долго не уменьшается (в течение двух эпох)
decay = keras.callbacks.ReduceLROnPlateau(monitor='loss', patience=2, factor=0.8, verbose=1)

# сохраняем модель после каждой эпохи, если лосс уменьшился
check = keras.callbacks.ModelCheckpoint(filepath=model_name+'.h5', monitor="loss")

In [71]:
# компилируем модель, используем оптимайзер Adam и triplet loss
opt = keras.optimizers.Adam(learning_rate=0.001)
model.compile(loss=triplet_loss, optimizer=opt)

In [72]:
# модель айтема
item_model().summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_8 (InputLayer)        [(None, 8888)]               0         []                            
                                                                                                  
 dense_7 (Dense)             (None, 512)                  4550656   ['input_8[0][0]']             
                                                                                                  
 dense_8 (Dense)             (None, 512)                  262144    ['dense_7[0][0]']             
                                                                                                  
 add_2 (Add)                 (None, 512)                  0         ['dense_7[0][0]',             
                                                                     'dense_8[0][0]']       

In [73]:
# модель юзера
user_model().summary()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_9 (InputLayer)        [(None, 19)]                 0         []                            
                                                                                                  
 dense_10 (Dense)            (None, 512)                  9728      ['input_9[0][0]']             
                                                                                                  
 dense_12 (Dense)            (None, 512)                  262144    ['dense_10[0][0]']            
                                                                                                  
 input_10 (InputLayer)       [(None, 9221)]               0         []                            
                                                                                            

In [74]:
# общая модель
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_4 (InputLayer)        [(None, 19)]                 0         []                            
                                                                                                  
 input_5 (InputLayer)        [(None, 9221)]               0         []                            
                                                                                                  
 input_6 (InputLayer)        [(None, 8888)]               0         []                            
                                                                                                  
 input_7 (InputLayer)        [(None, 8888)]               0         []                            
                                                                                            

## Обучение

In [75]:
# начинаем обучение, не забывая дропнуть столбцы item_id и user_id 
# из датафреймов при инициализации генератора.

# batch_size можно (и лучше) поставить побольше, если вы не органичены в ресурсах

model.fit(generator(items=items_ohe_train.drop(["item_id"], axis=1), 
                    users=users_ohe_train.drop(["user_id"], axis=1), 
                    interactions=interactions_vec_train,
                    batch_size=64), 
          steps_per_epoch=100,
          epochs=30,
          initial_epoch=0,
          callbacks=[decay, t_board, check]
)

Epoch 1/30
100/100 [==============================] - ETA: 0s - loss: 0.3303

C:\Users\BIT\.conda\envs\RecSys\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


100/100 [==============================] - 109s 1s/step - loss: 0.3303 - lr: 0.0010
Epoch 2/30
100/100 [==============================] - 109s 1s/step - loss: 0.2241 - lr: 0.0010
Epoch 3/30
100/100 [==============================] - 109s 1s/step - loss: 0.2101 - lr: 0.0010
Epoch 4/30
100/100 [==============================] - 108s 1s/step - loss: 0.1878 - lr: 0.0010
Epoch 5/30
100/100 [==============================] - 110s 1s/step - loss: 0.1711 - lr: 0.0010
Epoch 6/30
100/100 [==============================] - 111s 1s/step - loss: 0.1878 - lr: 0.0010
Epoch 7/30
100/100 [==============================] - 107s 1s/step - loss: 0.1695 - lr: 0.0010
Epoch 8/30
100/100 [==============================] - 106s 1s/step - loss: 0.1705 - lr: 0.0010
Epoch 9/30
100/100 [==============================] - 104s 1s/step - loss: 0.1598 - lr: 0.0010
Epoch 10/30
100/100 [==============================] - 104s 1s/step - loss: 0.1621 - lr: 0.0010
Epoch 11/30
100/100 [==============================] - 108s 

# Загрузка модели

In [93]:
save_model = keras.models.load_model(filepath="recsys_resnet_linear.h5", custom_objects={'triplet_loss': triplet_loss})

In [94]:
u2v = save_model.get_layer('model_1')

In [95]:
i2v = save_model.get_layer('model')

## Инференс на тёплых пользователях

In [96]:
# получаем фичи всех айтемов
items_feats_hot_test = items_ohe_train.drop(["item_id"], axis=1).to_numpy()

In [97]:
# получаем векторы всех айтемов
items_vecs_hot_test = i2v.predict(items_feats_hot_test)

289/289 [==============================] - 2s 6ms/step


In [98]:
# получаем фичи всех юзеров
users_meta_feats_hot_test = users_ohe_hot_test.drop(["user_id"], axis=1).to_numpy()

# получаем интеракции из train всех юзеров из hot_test
interactions_vec_inference = interactions_vec_train[users_ohe_hot_test.index.map(uid_to_user_id_hot_test).map(user_id_to_uid_train)].copy()

In [99]:
# получаем вектора всех юзеров
users_vec_hot_test = u2v.predict([np.array(users_meta_feats_hot_test), 
                                  np.array(interactions_vec_inference)])

867/867 [==============================] - 5s 6ms/step


In [100]:
dists = cosine_distances(users_vec_hot_test, items_vecs_hot_test)

In [101]:
top10_iids = np.argsort(dists, axis=1)[:,:10]

In [102]:
top10_iids

array([[8580, 5413, 3580, ..., 8647, 1235, 4375],
       [8580, 5413, 4169, ..., 7239, 4375, 6547],
       [7732, 7221, 6570, ..., 7904, 1991,  306],
       ...,
       [ 406, 7250, 2285, ..., 4233, 5758, 6075],
       [6570, 7732, 7755, ..., 7154, 5199, 4505],
       [5175, 5413, 8580, ..., 3184, 2285, 4233]], dtype=int64)

In [103]:
top10_iids_item = [iid_to_item_id_train[iid] for iid in top10_iids.reshape(-1)]

In [106]:
top10_iids_item = np.array(top10_iids_item).reshape(top10_iids.shape)

In [108]:
df_dssm = pd.DataFrame(columns = ['user_id', 'item_id'])

In [109]:
df_dssm = pd.DataFrame({'user_id': [uid_to_user_id_hot_test[uid] for uid in users_ohe_hot_test.index]})

In [110]:
df_dssm['item_id'] = list(top10_iids_item)

In [91]:
df_dssm = df_dssm.explode('item_id')
df_dssm['rank'] = df_dssm.groupby('user_id').cumcount() + 1
df_dssm = df_dssm.groupby('user_id').agg({'item_id': list}).reset_index()

In [92]:
df_dssm = df_dssm.explode('item_id')
df_dssm['rank'] = df_dssm.groupby('user_id').cumcount() + 1

In [93]:
df_dssm[240:250]

,user_id,item_id,rank
24,930,6402,1
24,930,6626,2
24,930,12701,3
24,930,3229,4
24,930,2358,5
24,930,6351,6
24,930,14337,7
24,930,1844,8
24,930,8447,9
24,930,9728,10


In [94]:
interactions_hot_test[interactions_hot_test.user_id==930]

,user_id,item_id,last_watch_dt,total_dur,watched_pct,uid,iid
1021831,930,3542,2021-08-22,1863,33,24,1992


# Раcчёт метрик на тёплых пользователях

In [95]:
metric_values_hot_test = calc_metrics(
    metrics,
    reco=df_dssm,
    interactions=interactions_hot_test,
    prev_interactions=interactions_train,
    catalog=catalog,
)

In [96]:
pd.DataFrame(metric_values_hot_test, index=['DSSM'])

,Precision@1,Recall@1,Precision@5,Recall@5,Precision@10,Recall@10,NDCG@1,NDCG@5,NDCG@10,MAP@1,MAP@5,MAP@10,MeanInvUserFreq@1,MeanInvUserFreq@5,MeanInvUserFreq@10,Serendipity@1,Serendipity@5,Serendipity@10
DSSM,0.015695,0.005545,0.014288,0.024223,0.012365,0.040824,0.015695,0.014693,0.013218,0.005545,0.012012,0.014398,5.04937,5.338499,5.601253,0.000117,0.00012,0.000123


# Возможный офлайн инференс для сервиса

In [151]:
# загружаем фичи айтемов из трейна
items_feats = pd.read_pickle('items_features_dssm.pkl')

In [152]:
# получаем фичи айтемов из трейна
items_feats = items_feats.drop(["item_id"], axis=1).to_numpy()

In [153]:
with open('iid_to_item_id.pkl', 'rb') as file:
    iid_to_item_id = pickle.load(file)

In [154]:
with open('item_id_to_iid.pkl', 'rb') as file:
    item_id_to_iid = pickle.load(file)

In [155]:
# загружаем фичи юзеров
users_feats = pd.read_pickle('users_features_dssm.pkl')

In [156]:
# получаем фичи юзеров
users_feats = users_feats.drop(["user_id"], axis=1).to_numpy()

In [157]:
with open('user_id_to_uid.pkl', 'rb') as file:
    user_id_to_uid = pickle.load(file)

In [158]:
interactions = pd.read_csv('data_original/interactions.csv', parse_dates=["last_watch_dt"])
interactions['watched_pct'] = interactions['watched_pct'].astype(pd.Int8Dtype())
interactions = interactions[interactions.watched_pct > 0]
interactions = interactions[interactions.item_id.isin(item_id_to_iid.keys())]
interactions = interactions[interactions.item_id.isin(item_id_to_iid.keys())]
interactions["iid"] = interactions["item_id"].apply(lambda x: item_id_to_iid[x])

In [159]:
intersection_users = set(interactions.user_id).intersection(set(user_id_to_uid))
interactions = interactions[interactions.user_id.isin(intersection_users)]

In [160]:
interactions.reset_index(drop=True, inplace=True)

In [161]:
N_FACTORS = 512

ITEM_MODEL_SHAPE = (items_feats.shape[1], ) 
USER_META_MODEL_SHAPE = (users_feats.shape[1], )

USER_INTERACTION_MODEL_SHAPE = (len(item_id_to_iid.keys()), )

print(f"N_FACTORS: {N_FACTORS}")
print(f"ITEM_MODEL_SHAPE: {ITEM_MODEL_SHAPE}")
print(f"USER_META_MODEL_SHAPE: {USER_META_MODEL_SHAPE}")
print(f"USER_INTERACTION_MODEL_SHAPE: {USER_INTERACTION_MODEL_SHAPE}")

N_FACTORS: 512
ITEM_MODEL_SHAPE: (8888,)
USER_META_MODEL_SHAPE: (19,)
USER_INTERACTION_MODEL_SHAPE: (9221,)


In [162]:
def triplet_loss(y_true, y_pred, n_dims=N_FACTORS, alpha=0.4):
    # будем ожидать, что на вход функции прилетит три сконкатенированных 
    # вектора - вектор юзера и два вектора айтема
    anchor = y_pred[:, 0:n_dims]
    positive = y_pred[:, n_dims:n_dims*2]
    negative = y_pred[:, n_dims*2:n_dims*3]

    # считаем расстояния от вектора юзера до вектора хорошего айтема
    pos_dist = K.sum(K.square(anchor - positive), axis=1)
    # и до плохого
    neg_dist = K.sum(K.square(anchor - negative), axis=1)

    # считаем лосс
    basic_loss = pos_dist - neg_dist + alpha
    loss = K.maximum(basic_loss, 0.0) # возвращаем ноль, если лосс отрицательный
 
    return loss

In [163]:
save_model = keras.models.load_model(filepath="recsys_resnet_linear.h5", custom_objects={'triplet_loss': triplet_loss})

In [164]:
u2v = save_model.get_layer('model_1')

In [165]:
i2v = save_model.get_layer('model')

In [166]:
# получаем векторы айтемов из трейна
items_vecs = i2v.predict(items_feats)

289/289 [==============================] - 2s 6ms/step


In [167]:
def create_user_vec(user_id):
    user_interactions_vec = np.zeros((1,len(item_id_to_iid.keys())))

    user_interactions = interactions[interactions.user_id == user_id]
    
    for i, item_id in enumerate(user_interactions.iid):
        user_interactions_vec[0, item_id] += user_interactions.watched_pct.iat[i]
    
    return user_interactions_vec / user_interactions_vec.sum()

In [168]:
user_ids = []
recos = []

In [169]:
for user_id in tqdm(interactions.user_id.unique()):
    if user_id in user_id_to_uid.keys():
        user_interactions_vec = create_user_vec(user_id)
        user_vec = u2v.predict([np.array(users_feats[user_id_to_uid[user_id]]).reshape(1,-1),
                                np.array(user_interactions_vec)])
        dists = cosine_distances(user_vec, items_vecs)
        top10_iids = np.argsort(dists, axis=1)[:,:10]
        top10_iids_item = [iid_to_item_id[iid] for iid in top10_iids.reshape(-1)]
        user_ids.append(user_id)
        recos.append(top10_iids_item)

  0%|          | 0/669188 [00:00<?, ?it/s]

1/1 [==============================] - 0s 27ms/step



KeyboardInterrupt



In [ ]:
recos_df = pd.DataFrame({'user_id':user_ids, 'rec': recos})

In [ ]:
recos_df

Офлайн рекомендации не были выданы из-за долгой обработки

Это можно было бы исправить батчами, но возникли проблемы с реализацией внутрих uid для создания матрицы векторов юзеров в create_user_vec